# Exemplos de tools funcionais

Agora vamos abordar a criação de duas tools com utilidade real para você entender como seria todo o processo.

## Criando um tool de busca de temperatura

https://open-meteo.com/en/docs

In [15]:
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

In [16]:
import requests
import datetime
from langchain.agents import tool
from pydantic import BaseModel, Field

class RetornaTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')

@tool
def retorna_temperatura_atual(latitude: float, longitude: float):
    """Retorna a temperatura atual dada uma determinada coordenada"""
    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))
        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return temp_atual
    else:
        raise Exception(f'Request para a API {URL} falhou: {resposta.status_code}')

In [17]:
retorna_temperatura_atual

StructuredTool(name='retorna_temperatura_atual', description='Retorna a temperatura atual dada uma determinada coordenada', args_schema=<class 'langchain_core.utils.pydantic.retorna_temperatura_atual'>, func=<function retorna_temperatura_atual at 0x000001AFD43FDA80>)

In [18]:
retorna_temperatura_atual.args

{'latitude': {'title': 'Latitude', 'type': 'number'},
 'longitude': {'title': 'Longitude', 'type': 'number'}}

### Testando a tool

In [19]:
retorna_temperatura_atual.invoke({'latitude': -18.64, 'longitude': -48.17})

24.2

## Criando um tool de busca no wikipedia

In [20]:
import wikipedia
wikipedia.set_lang('pt')

@tool
def busca_wikipedia(query: str):
    """Faz uma busca na Wikipédia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f"Título da página: {titulo}\nResumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return "Busca não teve retorno"
    else:
        return '\n\n'.join(resumos)


In [21]:
busca_wikipedia.invoke({'query': 'God of War'})

'Título da página: God of War\nResumo: God of War é uma série de jogos eletrônicos de ação-aventura e hack and slash, criada por David Jaffe da Santa Monica Studio da Sony, e publicado primeiramente para os consoles PlayStation. Iniciada em 2005, exclusivamente no console PlayStation 2 (PS2), a série tornou-se carro-chefe para a marca PlayStation, consistindo em nove jogos lançados, e em várias plataformas até o momento. Baseada em mitologias distintas, o enredo principal da série segue Kratos, um guerreiro espartano que foi levado a matar sua família por seu antigo mestre, chamado de deus da guerra Ares. Isso desencadeia uma série de eventos que levaram às guerras contra os panteões mitológicos. A era da mitologia grega da série, mostra Kratos seguindo um caminho de vingança devido às maquinações dos deuses do Olimpo, enquanto a era da mitologia nórdica, que apresenta seu filho Atreus como um protagonista secundário, e mostra um Kratos diferente, cansado e mais velho em um caminho de 

### Testando a tool

In [22]:
from langchain.prompts import ChatPromptTemplate as cpt
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = cpt.from_messages([
    ('system', 'Você é um assitente amigável chamado Isaac'),
    ('user', '{input}')
])

chat = ChatOpenAI(model='gpt-4o-mini')

tools = [convert_to_openai_function(busca_wikipedia), convert_to_openai_function(retorna_temperatura_atual)]
chain = prompt | chat.bind(functions=tools)

In [23]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 98, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ddc3dea-1939-4c0c-960d-ab55cf2a9e30-0', usage_metadata={'input_tokens': 98, 'output_tokens': 9, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
chain.invoke({'input': 'Qual a temperatura de Catalão Goiás?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-18.1765,"longitude":-47.9381}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 104, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-bcf5fa53-045b-4a9a-b5dd-5bd714f0ff13-0', usage_metadata={'input_tokens': 104, 'output_tokens': 32, 'total_tokens': 136, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [25]:
chain.invoke({'input': 'O que é God of War?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"God of War"}', 'name': 'busca_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 104, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-4a519c00-b1f1-4fab-a251-5970b03e878c-0', usage_metadata={'input_tokens': 104, 'output_tokens': 18, 'total_tokens': 122, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})